In [ ]:
!pip install transformers

In [ ]:
from PIL import Image
from transformers import AutoProcessor, BlipForQuestionAnswering
import os
import cv2
from tqdm import tqdm
import torch
from transformers import AutoModelForCausalLM
from google.colab.patches import cv2_imshow

In [ ]:
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = AutoProcessor.from_pretrained("Salesforce/blip-vqa-base")

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
!mkdir blip_output

In [ ]:
directory_path = '/content/videos'
directory_files = os.listdir(directory_path)
#video_list = ['1.mp4','2.mp4','3.mp4','4.mp4']
#The input and outputAppend sentences must match.
#"What is the make and model of the vehicle in the provided image?",  "What is the Make/model of the damaged car in the provided image?",
inputSentences = ["In which area did this accident happen?", "What is the time of the day when this accident happened?", "What is the weather condition in this accident?",
                  "What is the season of",
                  "How many vehicles are in involved in this accident?", "What type of vehicles are in the provided image?",
                  "What is the damaged vehicle in the provided image?",
                  "Which side of vehicle is damaged in the provided image? Front or Back?"]
outputAppendText = ["The accident occurred on ", "The accident took place at ",
                    "The accident happened under weather condition, ", "The incident took place during ",
                    "The number of vehicles are ",
                    "The road had ", "The vehicle involved in the accident was ",
                    "The vehicle sustained damage on its "]
#outputSentence = []
for video in directory_files:
  #outputText = []
  outputText = set()
  array_of_dict = [{},{},{},{}]
  print(video)
  # Create a VideoCapture object and read from input file
  cap = cv2.VideoCapture("/content/videos/" + video)

  # Get the frame rate of the video
  fps = cap.get(cv2.CAP_PROP_FPS)

  # Define the desired frame interval (e.g., every second)
  frame_interval = int(fps)  # Extract one frame per second

  # Get the total number of frames in the video
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

  # Create a tqdm progress bar
  progress_bar = tqdm(total=total_frames, desc='Processing Frames', unit='frame')

  if (cap.isOpened()== False):
  # Give a error message
    print("Error opening video file")

  progress = 0
  frame_count = 0
  final_caption = ""
  final_frames_output = ""
  # Read until video is completed
  while(cap.isOpened()):

  # Capture frame-by-frame
      ret, frame = cap.read()
      if ret == True:
        progress_bar.update(1)
        progress = progress + 1
        if(progress > total_frames):
          break
        frame_count = frame_count + 1
        if frame_count%(frame_interval) == 0:
          # process image to text
          device = "cuda" if torch.cuda.is_available() else "cpu"
          pixel_values = processor(images=frame, return_tensors="pt").pixel_values
          #cv2_imshow(frame)
          #outputFrameCount = 0
          frame_output = ""
          for i in range(0, len(inputSentences)):
            #outputFrameCount = 1
            inputs = processor(images=frame, text=inputSentences[i], return_tensors="pt")
            outputs = model.generate(**inputs)
            #print(processor.decode(outputs[0], skip_special_tokens=True))
            outputLabel = processor.decode(outputs[0], skip_special_tokens=True)
            if i <= 3:
              if outputLabel not in array_of_dict[i]:
                array_of_dict[i][outputLabel] = 0
              array_of_dict[i][outputLabel] = array_of_dict[i][outputLabel] + 1
            else:
              if outputLabel == 'none' or outputLabel == 'no' or outputLabel == 'no vehicle' or outputLabel == 0 or outputLabel == '0' or outputLabel == 'no car' or outputLabel == 1 or outputLabel == '1':
                print("")
              else:
                outputText.add(outputAppendText[i] + outputLabel + ".")
                frame_output = frame_output + outputAppendText[i] + outputLabel + "."
            #if(outputFrameCount <= 1):
              #if not outputText:
                #new_dict = {outputLabel: 1}
              #else:
                #new_dict = {outputLabel: 1}
              #outputText.append(new_dict)
            #else:
              #for dictionary in outputText:
                #if outputLabel not in dictionary:
                  #dictionary[outputLabel] = 0
                #dictionary[outputLabel] = dictionary[outputLabel] + 1
            #final_caption = final_caption + generated_caption
            cv2.waitKey(0)

          final_frames_output = final_frames_output + "frame " + str(frame_count) + ", " + frame_output
      # Press Q on keyboard to exit
          if cv2.waitKey(25) & 0xFF == ord('q'):
              break

  # Break the loop
      else:
          break
  #print(final_caption)
  #Iterate through each dictionary and find the key with the highest value
  if array_of_dict:
    print("array_of_dict")
    print(array_of_dict)
    outputLabelText = [max(d, key=d.get) for d in array_of_dict if d]
    for text, appendText in zip(outputLabelText, outputAppendText):
      outputText.add(appendText + text + ".")
      final_frames_output = final_frames_output + "common for all frames, " + appendText + text + "."
  # When everything done, release
  # the video capture object
  cap.release()

  # Closes all the frames
  cv2.destroyAllWindows()
  # Iterate through the dictionaries in the input list
  #for dictionary in outputText:
      # Remove the "none" key if it exists
  #    dictionary.pop("none", None)

  # Iterate through each dictionary and find the key with the highest value
  #outputLabelText = [max(d, key=d.get) if d else None for d in outputText]
  #for text, appendText in zip(outputLabelText, outputAppendText):
  #  outputSentence.append(appendText + text)
  #print('\n')
  #print(''.join(outputSentence))
  print(outputText)
  f = open("/content/blip_output/" + video.split(".")[0] + ".txt", "w")
  f.write(final_frames_output)
  f.close()
  print(final_frames_output)

CLIP_322.mp4


Processing Frames:   0%|          | 1/1348 [00:00<03:36,  6.23frame/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Processing Frames:   4%|▍         | 59/1348 [00:22<06:55,  3.10frame/s]

Processing Frames:   4%|▍         | 60/1348 [00:38<06:55,  3.10frame/s]

Processing Frames:   7%|▋         | 90/1348 [00:58<09:58,  2.10frame/s]

...

Processing Frames:  86%|████████▋ | 769/890 [12:02<01:58,  1.02frame/s]

Processing Frames:  89%|████████▉ | 793/890 [12:25<01:35,  1.02frame/s]

Processing Frames:  94%|█████████▎| 834/890 [12:47<00:40,  1.39frame/s]

Processing Frames:  94%|█████████▍| 841/890 [13:10<00:51,  1.05s/frame]

array_of_dict
[{'washington': 4, 'highway': 30, 'airport': 1}, {'7 : 00 am': 13, '6 : 00 am': 1, '6 : 00 pm': 5, 'evening': 3, 'late': 1, '3 : 00 pm': 4, 'late afternoon': 7, 'night': 1}, {'stormy': 14, 'rainy': 17, 'cloudy': 4}, {'fall': 28, 'summer': 4, 'winter': 3}]
{'The vehicle involved in the accident was truck.', 'The road had cars.', 'The incident took place during fall.', 'The road had motorcycles.', 'The vehicle sustained damage on its back.', 'The accident happened under weather condition, rainy.', 'The accident occurred on highway.', 'The vehicle involved in the accident was car.', 'The road had truck.', 'The vehicle involved in the accident was bus.', 'The number of vehicles are 3.', 'The accident took place at 7 : 00 am.', 'The road had vans.', 'The road had trucks.', 'The vehicle involved in the accident was suv.', 'The number of vehicles are 2.'}
frame 24, The number of vehicles are 2.The road had cars.The vehicle involved in the accident was suv.The vehicle sustained d

In [ ]:
!zip -r /content/blip_output.zip /content/blip_output/

  adding: content/blip_output/ (stored 0%)
  adding: content/blip_output/CLIP_286.txt (deflated 91%)
  adding: content/blip_output/CLIP_324.txt (deflated 90%)
  adding: content/blip_output/CLIP_301.txt (deflated 91%)
  adding: content/blip_output/.ipynb_checkpoints/ (stored 0%)
  adding: content/blip_output/CLIP_312.txt (deflated 88%)
